In [4]:
import pandas as pd 
import math

In [5]:
def generate_combinations(arr, r):
    """
    Generates combinations of length 'r' from the given array 'arr'
    """
    n = len(arr)
    # Base case: if r is 0 or greater than length of arr
    if r == 0 or r > n:
        return [[]]
    # Base case: if r is equal to length of arr
    if r == n:
        return [arr]

    # Recursive case
    combinations = []

    # Generate combinations including the first element
    for comb in generate_combinations(arr[1:], r - 1):
        combinations.append([arr[0]] + comb)

    # Generate combinations excluding the first element
    combinations.extend(generate_combinations(arr[1:], r))

    return combinations

In [6]:
def contains(larger, smaller):
  larger_iter = iter(larger)
  return all(s in larger_iter for s in smaller)

In [7]:
def remove_same_col(combinations):
    new=[]
    for i in range(len(combinations)) :
        combinations[i].sort()
        number ="999"
        flag=1
        for j in combinations[i]:
            if(j[0]== number):
                flag=0
                break
            else:
                number=j[0]
        if flag:
            new.append(combinations[i])
    return new

In [8]:
#combination of l 
def com_PL(L_initial,prev_to_remove,num):
    l1=[]
    for i in range(len(L_initial)):
        for j in range(i+1,len(L_initial)):
            list=[]
            all=[]
            list.extend(L_initial[i])
            list.extend(L_initial[j])
            all = generate_combinations(list,num)
            if(num>1):
                all = remove_same_col(all)
                #print(all)
                #print("done")
            #all.unique()
            for com in all:
                add = 1 
                #search in the non frequent set
                for k in prev_to_remove:
                    if(contains(com,k)):
                        add=0
                        break

                if add:
                    for kok in l1:
                            if(contains(kok,com)):
                                add=0
                                break
                if (add):   
                    l1.extend([com])
    return l1

In [9]:
# count for Ls
def trans_count(combinations,trans,msl):
    to_remove=[]
    L_initial=[]
    L_initial_pair=[]
    flag=0
    for com in combinations :
        count=0    
        for row in trans:
            if(len(com)>len(row)):
                flag=0
                return to_remove,L_initial,L_initial_pair,flag
            
            if contains(row,com):
                count+=1
        if(count>=msl):
            L_initial_pair.append([com, count])
            #print(com)
            L_initial.append(com)
            flag=1
        else:
            #for the next L
            to_remove.append(com)
            
    return to_remove,L_initial,L_initial_pair,flag

In [10]:
def frequentItemsSets(unique_values_per_column,trans,ms):
    Ls=[]
    count=1
    flag =1
    prev_l=unique_values_per_column
    prev_to_remove=[]
    while(flag):
        #print(count)
        prev_l =com_PL(prev_l,prev_to_remove,count)
        #print(prev_l)
        prev_to_remove,prev_l,L,flag =trans_count(prev_l,trans,ms)
        #print(prev_l)
        if(flag):
            Ls.append(L)
            count+=1
    return Ls

In [41]:
def confidence(freqsets,mcl):
    all_conf=[]
    for i in range(1,len(freqsets)):
        current=freqsets[i]
        #loop over each item
        for pair in current:
            list=pair[0]
            count=pair[1]
            bcount=len(pair[0])-1
            for fcount in range(1,len(list)):
                # if(fcount>bcount):
                #     break
                combs = generate_combinations(list,fcount)
                for comb in combs:
                    for pp in freqsets[len(comb)-1]:
                        if(contains(comb,pp[0])):
                            conf=count/pp[1]
                            if(conf >= mcl):
                                difference = [item for item in list if item not in pp[0]]
                                all_conf.append([pp[0],difference,round(conf, 2)])
                            break
    return all_conf

In [ ]:
def confidence(freqsets,mcl):
    all_conf=[]
    for i in range(1,len(freqsets)):
        current=freqsets[i]
        #loop over each item
        for pair in current:
            list=pair[0]
            count=pair[1]
            bcount=len(pair[0])-1
            for fcount in range(int(len(list)/2)):
                combs = generate_combinations(list,fcount+1)
                for comb in combs:
                    for pp in freqsets[len(comb)-1]:
                        if(contains(comb,pp[0])):
                            conf=count/pp[1]
                            if(conf >= mcl ):
                                difference = [item for item in list if item not in pp[0]]
                                all_conf.append([pp[0],difference,round(conf, 2)])
                            break
    return all_conf

In [35]:
def apriori(msl,mcl,path):
    data = pd.read_csv(path)
    data.drop(columns=["TransactionNo","DateTime"],inplace=True)

    #for adding col number to remove the combinations 
    for i, column in enumerate(data.columns):
        data[column] = str(i) + data[column].astype(str)

    # combining all the transactions into a list of lists 
    tran=[]
    for _, row in data.iterrows():
       tran.append(row.to_list())

    # Combine all the unique values into a list
    unique_values_per_column = []
    for column_name in data.columns:
        uni=data[column_name].unique().tolist()
        unique_values_per_column.extend(uni)
    for i in range(len(unique_values_per_column)):
        unique_values_per_column[i]= [unique_values_per_column[i]]

    ms= math.ceil(msl*len(tran))
    print(ms)
    Freqentsets = frequentItemsSets(unique_values_per_column,tran,ms)

    all_conff=confidence(Freqentsets,mcl)
        
    
    return all_conff

In [39]:
sets=apriori(.001,.5,"/Users/faressaad/Downloads/Assignment1/Bakery.csv")

21


In [21]:
print(sets)

[[['0Bread'], ['2Weekday'], 0.63], [['0Scandinavian'], ['2Weekend'], 0.52], [['0Hot chocolate'], ['1Afternoon'], 0.57], [['0Hot chocolate'], ['2Weekday'], 0.58], [['0Jam'], ['1Morning'], 0.54], [['0Jam'], ['2Weekday'], 0.58], [['0Cookies'], ['1Afternoon'], 0.58], [['0Cookies'], ['2Weekday'], 0.74], [['0Muffin'], ['1Afternoon'], 0.56], [['0Muffin'], ['2Weekday'], 0.58], [['0Coffee'], ['1Afternoon'], 0.52], [['0Coffee'], ['2Weekday'], 0.65], [['0Pastry'], ['1Morning'], 0.71], [['0Pastry'], ['2Weekday'], 0.66], [['0Medialuna'], ['1Morning'], 0.65], [['0Medialuna'], ['2Weekday'], 0.55], [['0Tea'], ['1Afternoon'], 0.65], [['0Tea'], ['2Weekday'], 0.68], [['0Tartine'], ['1Afternoon'], 0.57], [['0Tartine'], ['2Weekday'], 0.51], [['0Mineral water'], ['1Afternoon'], 0.82], [['0Mineral water'], ['2Weekday'], 0.63], [['0Farm House'], ['1Morning'], 0.56], [['0Farm House'], ['2Weekday'], 0.66], [['0Fudge'], ['1Afternoon'], 0.57], [['0Fudge'], ['2Weekday'], 0.62], [['0Juice'], ['1Afternoon'], 0.61], 

In [40]:
print(sets)

[[['0Bread'], ['2Weekday'], 0.63], [['0Scandinavian'], ['2Weekend'], 0.52], [['0Hot chocolate'], ['1Afternoon'], 0.57], [['0Hot chocolate'], ['2Weekday'], 0.58], [['0Jam'], ['1Morning'], 0.54], [['0Jam'], ['2Weekday'], 0.58], [['0Cookies'], ['1Afternoon'], 0.58], [['0Cookies'], ['2Weekday'], 0.74], [['0Muffin'], ['1Afternoon'], 0.56], [['0Muffin'], ['2Weekday'], 0.58], [['0Coffee'], ['1Afternoon'], 0.52], [['0Coffee'], ['2Weekday'], 0.65], [['0Pastry'], ['1Morning'], 0.71], [['0Pastry'], ['2Weekday'], 0.66], [['0Medialuna'], ['1Morning'], 0.65], [['0Medialuna'], ['2Weekday'], 0.55], [['0Tea'], ['1Afternoon'], 0.65], [['0Tea'], ['2Weekday'], 0.68], [['0Tartine'], ['1Afternoon'], 0.57], [['0Tartine'], ['2Weekday'], 0.51], [['0Mineral water'], ['1Afternoon'], 0.82], [['0Mineral water'], ['2Weekday'], 0.63], [['0Farm House'], ['1Morning'], 0.56], [['0Farm House'], ['2Weekday'], 0.66], [['0Fudge'], ['1Afternoon'], 0.57], [['0Fudge'], ['2Weekday'], 0.62], [['0Juice'], ['1Afternoon'], 0.61], 

In [20]:
print(fes)

['1Morning', '3Weekend', '5Bread']
